# Fx Forward Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-fx-cross-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [6]:
import refinitiv.dataplatform as rdp
import pandas as pd

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Pricing use case

This sample computes a cross forward at a specific broken date and returns the intermediary information used to derive the value :
- the interpolated forward for each currency ('FxSwapsCcy1', 'FxSwapsCcy2')
- the cross spot rate ('FxSpot')
- the cross forward rate ('FxSwapsCcy1Ccy2')
- the cross outright rate ('FxOutrightCcy1Ccy2')

it also provides information such as athe scaling factor used for the swap points ('FxSwapPointScalingFactor') and the spot rate ('FxCrossScalingFactor') or the actual start date ('StartDate') and end date ('EndDate') of the contract.



In [5]:
request_body = {
    
    "fields":["InstrumentDescription","FxCrossScalingFactor","FxSwapPointScalingFactor",\
            "ValuationDate","StartDate","EndDate","Tenor",\
            "FxSpot","FxSwapsCcy1","FxSwapsCcy2","FxSwapsCcy1Ccy2","FxOutrightCcy1Ccy2",\
            "ErrorCode","ErrorMessage"],
    
    "universe":[
        {
            "instrumentType" : "FxCross",
            "instrumentDefinition" : {
                "instrumentTag" : "Fx-Forward",
                "fxCrossType" : "FxForward",
                "fxCrossCode" : "KRWINR",
                "legs" : [
                    {
                        "endDate" : "2020-12-02",
                        #"tenor":"1M",
                    }]

            }
        }
    ],
    
    "pricingParameters":{
        "valuationDate":"2020-04-15", #by default today is used
        "ignoreRefCcyHolidays":False,
        "priceSide":"ask"
    },
    
    "outputs" : ["Data","Headers"]
   
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentDescription,FxCrossScalingFactor,FxSwapPointScalingFactor,ValuationDate,StartDate,EndDate,Tenor,FxSpot,FxSwapsCcy1,FxSwapsCcy2,FxSwapsCcy1Ccy2,FxOutrightCcy1Ccy2,ErrorCode,ErrorMessage
0,FxForward KRWINR 2020-12-2,1.0,10000.0,2020-04-15T00:00:00Z,2020-04-17T00:00:00Z,2020-12-02T00:00:00Z,None,0.062521,-758.695652,204.3286,21.48954,0.06467,,
